In [1]:
import torch
from transformers import Qwen2_5_VLForConditionalGeneration, AutoProcessor
from qwen_vl_utils import process_vision_info  # pip install qwen-vl-utils
import json
import os
from PIL import Image

# --- 설정 (사용자 환경에 맞게 수정) ---
# JSON 파일과 프레임 폴더 경로 설정
json_file_path = '/home/etri/DHL/Data/test_data/건물진입_tracking.json'
frame_folder_path = '/home/etri/DHL/Data/test_data/output_frames_1'
video_name = '건물진입'
output_json_path = '/home/etri/DHL/SGG+LLM/output_relation/qwen_output_relations.json'

# 비디오 전체 문맥 정보
video_captioning_info = "A group of soldiers is seen running down a deserted street in a tropical setting. The environment features palm trees, small buildings with flat roofs, and a dirt hillside in the background. A blue car is parked on the side of the road, and a red barrier is placed near."

# --- 프롬프트 템플릿 (기존 프롬프트 활용) ---
# 첫 번째 프레임용 프롬프트
base_prompt = """
You are an expert in video analysis and extracting scene graph triplets for military and tactical scenarios.
Your primary goal is to extract the maximum number of static relationships possible between all detected objects. You must consider every plausible static relationship for all object pairs.
Your task is to identify ALL relevant static relationships between these objects and output them as a single JSON object containing two separate lists: `static_relations` and `dynamic_relations`.
The 'predicate' should describe the relationship accurately based on the visual context, the detected objects, and the provided captioning information.

This is the very first frame of the video (Frame {frame_number} out of {total_frames}).
Here are the detected objects in this frame:
{objects_str}

Video Captioning Information (Overall Context): "{video_captioning_info}"

ALLOWED OBJECTS:
tree,soldier,car,building

ALLOWED PREDICATES:
1. Static Relations: next to, in front of, behind, on top of, under, covered by, inside, outside, between, blocking, wearing, parked, stationary, has, damaged, destroyed, abandoned, occupied, unoccupied, intact
2. Dynamic Relations: enter, exit, patrol, explode, sail, fire, cross, load, unload, land, take off, pass by, gather, circle

CRITICAL RULES:
1. STRICT ID-Label Consistency: The `subject_id` and `object_id` MUST PERFECTLY MATCH the `track_id`. `subject_label` and `object_label` MUST MATCH the `label`.
2. Categorized Output: The final output MUST be a JSON object with two keys: `static_relations` and `dynamic_relations`. If none, output empty lists.
3. For this first frame, only `static_relations` should be populated.

Example of a valid JSON output:

{{
"static_relations": [
{{
"subject_id": "2", "predicate": "next to", "object_id": "1",
"subject_label": "soldier", "object_label": "helicopter"
}}
],
"dynamic_relations": []
}}
"""


# 연속된 두 프레임 비교를 위한 프롬프트
modified_prompt = """
You are an expert in video analysis and extracting scene graph triplets for military and tactical scenarios.
Your task is to analyze two consecutive frames from a video to identify both static and dynamic relationships between objects.
The first image is the 'previous frame' and the second image is the 'current frame'.
Your primary goal is to infer dynamic relations by comparing the object movements between these two frames. Also, update the static relations based on the current frame.

Here is the context:
- Previous Frame Objects: {previous_objects_str}
- Current Frame Objects (Frame {frame_number}): {objects_str}
- Video Captioning Information (Overall Context): "{video_captioning_info}"

ALLOWED OBJECTS:
tree,soldier,car,building

ALLOWED PREDICATES:
1. Static Relations: next to, in front of, behind, on top of, under, covered by, inside, outside, between, blocking, wearing, parked, stationary, has, damaged, destroyed, abandoned, occupied, unoccupied, intact
2. Dynamic Relations: enter, exit, patrol, explode, sail, fire, cross, load, unload, land, take off, pass by, gather, circle

CRITICAL RULES:
1. Dynamic Relation Inference: Infer dynamic predicates by analyzing changes between the previous and current frames (e.g., bounding box crossing for 'enter'/'exit', trajectory changes for 'pass by'). Consider all possible pairs.
2. Static Relation Persistence: Assume previous static relations persist unless there is a clear change in the current frame.
3. Logical Subject-Object Order: For dynamic predicates, the 'subject' MUST be the moving object.
4. STRICT ID-Label Consistency: IDs and labels in relations MUST PERFECTLY MATCH the provided object lists.
5. Categorized Output: The final output MUST be a JSON object with two keys: `static_relations` and `dynamic_relations`.

Example of a valid JSON output:
{{
"static_relations": [
{{"subject_id": "5", "predicate": "has", "object_id": "6", "subject_label": "soldier", "object_label": "weapon"}}
],
"dynamic_relations": [
{{"subject_id": "2", "predicate": "enter", "object_id": "1", "subject_label": "soldier", "object_label": "helicopter"}}
]
}}
"""

/home/etri/miniconda3/envs/test_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
import torch
from transformers import Qwen2_5_VLForConditionalGeneration, AutoProcessor
from qwen_vl_utils import process_vision_info
import json
import os
from PIL import Image

# --- 설정 (사용자 환경에 맞게 수정) ---
json_file_path = '/home/etri/DHL/Data/test_data/건물진입_tracking.json'
frame_folder_path = '/home/etri/DHL/Data/test_data/output_frames_1'
video_name = '건물진입'
output_json_path = '/home/etri/DHL/SGG+LLM/output_relation/qwen_output_relations_final.json'
video_captioning_info = "A group of soldiers is seen running down a deserted street in a tropical setting. The environment features palm trees, small buildings with flat roofs, and a dirt hillside in the background. A blue car is parked on the side of the road, and a red barrier is placed near."

# --- 프롬프트 템플릿 (기존과 동일) ---
base_prompt = """
You are an expert in video analysis and extracting scene graph triplets for military and tactical scenarios.
Your primary goal is to extract the maximum number of static relationships possible between all detected objects. You must consider every plausible static relationship for all object pairs.
Your task is to identify ALL relevant static relationships between these objects and output them as a single JSON object containing two separate lists: `static_relations` and `dynamic_relations`.
The 'predicate' should describe the relationship accurately based on the visual context, the detected objects, and the provided captioning information.

This is the very first frame of the video (Frame {frame_number} out of {total_frames}).
Here are the detected objects in this frame:
{objects_str}

Video Captioning Information (Overall Context): "{video_captioning_info}"

ALLOWED OBJECTS:
tree,soldier,car,building

ALLOWED PREDICATES:
1. Static Relations: next to, in front of, behind, on top of, under, covered by, inside, outside, between, blocking, wearing, parked, stationary, has, damaged, destroyed, abandoned, occupied, unoccupied, intact
2. Dynamic Relations: enter, exit, pass by

CRITICAL RULES:
1. STRICT ID-Label Consistency: The `subject_id` and `object_id` MUST PERFECTLY MATCH the `track_id`. `subject_label` and `object_label` MUST MATCH the `label`.
2. Categorized Output: The final output MUST be a JSON object with two keys: `static_relations` and `dynamic_relations`. If none, output empty lists.
3. For this first frame, only `static_relations` should be populated.

Example of a valid JSON output:

{{
"static_relations": [
{{
"subject_id": "2", "predicate": "next to", "object_id": "1",
"subject_label": "soldier", "object_label": "helicopter"
}}
],
"dynamic_relations": ["subject_id": "2", "predicate": "enter", "object_id": "3",
"subject_label": "soldier", "object_label": "building"]
}}



TASK
1. From the provided frame(s) and object list, output a JSON object with two keys only:
   • "static_relations"    - max 5 items  
   • "dynamic_relations"   - max 5 items  
2. Rank candidate relations by visual certainty and overall relevance, then keep only the top five in each list.
3. Follow these formatting rules exactly:
   • Each relation must be an object with the keys:
     "subject_id", "predicate", "object_id", "subject_label", "object_label"
   • If fewer than five valid relations exist, return only those found.
   • Do not add any keys or metadata outside the required JSON structure.
4. Allowed predicates and objects remain the same as previously specified.

Return **only** the final valid JSON
"""

modified_prompt = """
You are an expert in video analysis and extracting scene graph triplets for military and tactical scenarios.
Your task is to analyze two consecutive frames from a video to identify both static and dynamic relationships between objects.
The first image is the 'previous frame' and the second image is the 'current frame'.
Your primary goal is to infer dynamic relations by comparing the object movements between these two frames. Also, update the static relations based on the current frame.

Here is the context:
- Previous Frame Objects: {previous_objects_str}
- Current Frame Objects (Frame {frame_number}): {objects_str}
- Video Captioning Information (Overall Context): "{video_captioning_info}"

ALLOWED OBJECTS:
tree,soldier,car,building

ALLOWED PREDICATES:
1. Static Relations: next to, in front of, behind, on top of, under, covered by, inside, outside, between, blocking, wearing, parked, stationary, has, damaged, destroyed, abandoned, occupied, unoccupied, intact
2. Dynamic Relations: enter, exit, pass by

CRITICAL RULES:
1. Dynamic Relation Inference: Infer dynamic predicates by analyzing changes between the previous and current frames (e.g., bounding box crossing for 'enter'/'exit', trajectory changes for 'pass by'). 
   Consider all possible pairs.
2. Static Relation Persistence: Assume previous static relations persist unless there is a clear change in the current frame.
3. Logical Subject-Object Order: For dynamic predicates, the 'subject' MUST be the moving object.
4. STRICT ID-Label Consistency: IDs and labels in relations MUST PERFECTLY MATCH the provided object lists.
5. Categorized Output: The final output MUST be a JSON object with two keys: `static_relations` and `dynamic_relations`.
6. Focus on the soldiers' movement

Example of a valid JSON output:
{{
"static_relations": [
{{"subject_id": "5", "predicate": "has", "object_id": "6", "subject_label": "soldier", "object_label": "weapon"}}
],
"dynamic_relations": [
{{"subject_id": "2", "predicate": "enter", "object_id": "1", "subject_label": "soldier", "object_label": "helicopter"}}
]
}}

TASK
1. From the provided frame(s) and object list, output a JSON object with two keys only:
   • "static_relations"   – max 5 items  
   • "dynamic_relations" – max 5 items  
2. Rank candidate relations by visual certainty and overall relevance, then keep only the top five in each list.
3. Follow these formatting rules exactly:
   • Each relation must be an object with the keys:
     "subject_id", "predicate", "object_id", "subject_label", "object_label"
   • If fewer than five valid relations exist, return only those found.
   • Do not add any keys or metadata outside the required JSON structure.
4. Allowed predicates and objects remain the same as previously specified.

Return **only** the final valid JSON
"""

/home/etri/miniconda3/envs/test_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
import torch
from transformers import Qwen2_5_VLForConditionalGeneration, AutoProcessor
from qwen_vl_utils import process_vision_info
import json
import os
from PIL import Image

# --- 설정 (사용자 환경에 맞게 수정) ---
json_file_path = '/home/etri/DHL/Data/test_data/건물진입_tracking.json'
frame_folder_path = '/home/etri/DHL/Data/test_data/output_frames_1'
video_name = '건물진입'
output_json_path = '/home/etri/DHL/SGG+LLM/output_relation/qwen_output_relations_final_final.json'
video_captioning_info = "A group of soldiers is seen running down a deserted street in a tropical setting. The environment features palm trees, small buildings with flat roofs, and a dirt hillside in the background. A blue car is parked on the side of the road, and a red barrier is placed near."

# --- 프롬프트 템플릿 (JSON 예시 이스케이프 적용) ---
base_prompt = """
You are an expert in video analysis and extracting scene graph triplets for military and tactical scenarios.
Your primary goal is to extract the maximum number of static relationships possible between all detected objects. You must consider every plausible static relationship for all object pairs.
Your task is to identify ALL relevant static relationships between these objects and output them as a single JSON object containing two separate lists: `static_relations` and `dynamic_relations`.
The 'predicate' should describe the relationship accurately based on the visual context, the detected objects, and the provided captioning information.

This is the very first frame of the video (Frame {frame_number} out of {total_frames}).
Here are the detected objects in this frame:
{objects_str}

Video Captioning Information (Overall Context): "{video_captioning_info}"

ALLOWED OBJECTS:
tree,soldier,car,building

ALLOWED PREDICATES:
1. Static Relations: next to, in front of, on top of, under, covered by, inside, outside, between
2. Dynamic Relations: enter, exit, pass by

CRITICAL RULES:
1. STRICT ID-Label Consistency: The `subject_id` and `object_id` MUST PERFECTLY MATCH the `track_id`. `subject_label` and `object_label` MUST MATCH the `label`.
2. Categorized Output: The final output MUST be a JSON object with two keys: `static_relations` and `dynamic_relations`. If none, output empty lists.
3. For this first frame, only `static_relations` should be populated.

Example of a valid JSON output:

{{
  "static_relations": [
    {{
      "subject_id": "2", "predicate": "next to", "object_id": "1",
      "subject_label": "soldier", "object_label": "helicopter"
    }}
  ],
  "dynamic_relations": [
    {{
      "subject_id": "2", "predicate": "enter", "object_id": "3",
      "subject_label": "soldier", "object_label": "building"
    }}
  ]
}}

TASK
1. From the provided frame(s) and object list, output a JSON object with two keys only:
   • "static_relations"    - max 5 items  
   • "dynamic_relations"   - max 5 items  
2. Rank candidate relations by visual certainty and overall relevance, then keep only the top five in each list.
3. Follow these formatting rules exactly:
   • Each relation must be an object with the keys:
     "subject_id", "predicate", "object_id", "subject_label", "object_label"
   • If fewer than five valid relations exist, return only those found.
   • Do not add any keys or metadata outside the required JSON structure.
4. Allowed predicates and objects remain the same as previously specified.

Return **only** the final valid JSON
"""

modified_prompt = """
You are an expert in video analysis and extracting scene graph triplets for military and tactical scenarios.
Your task is to analyze two consecutive frames from a video to identify both static and dynamic relationships between objects.
The first image is the 'previous frame' and the second image is the 'current frame'.
Your primary goal is to infer dynamic relations by comparing the object movements between these two frames. Also, update the static relations based on the current frame.

Here is the context:
- Previous Frame Objects: {previous_objects_str}
- Current Frame Objects (Frame {frame_number}): {objects_str}
- Video Captioning Information (Overall Context): "{video_captioning_info}"

ALLOWED OBJECTS:
tree,soldier,car,building

ALLOWED PREDICATES:
1. Static Relations: next to, in front of, on top of, under, covered by, inside, outside, between
2. Dynamic Relations: enter, exit, pass by

STATIC RELATIONS GUIDELINES:
- For positional static relations (e.g., next to, in front of, behind, on top of, under, inside, outside, between), ONLY extract them for nearby object pairs.
- "Nearby" means: Objects with bounding boxes that are adjacent, overlapping, or within a small distance (e.g., centers closer than 20% of image width/height).
- Do NOT extract positional relations for distant objects; focus on visually close pairs to avoid irrelevant relations.
- Example: Extract "soldier next to building" only if the soldier's bounding box is adjacent to the building's.

DYNAMIC RELATIONS DEFINITIONS & EXAMPLES:
• "enter": Object moves INTO another object's boundary or space
  Example: soldier's bounding box overlaps or crosses into building's bounding box
  {{"subject_id": "1", "predicate": "enter", "object_id": "2", "subject_label": "soldier", "object_label": "building"}}

• "exit": Object moves OUT OF another object's boundary or space  
  Example: soldier's bounding box moves away from or stops overlapping with building's bounding box
  {{"subject_id": "1", "predicate": "exit", "object_id": "2", "subject_label": "soldier", "object_label": "building"}}

• "pass by": Object moves NEAR another object without entering
  Example: soldier's trajectory passes close to a car but doesn't overlap
  {{"subject_id": "1", "predicate": "pass by", "object_id": "3", "subject_label": "soldier", "object_label": "building"}}
  {{"subject_id": "1", "predicate": "pass by", "object_id": "3", "subject_label": "soldier", "object_label": "car"}}

MOTION ANALYSIS GUIDELINES:
- Calculate object center displacement: dx = current_center_x - previous_center_x, dy = current_center_y - previous_center_y
- Check bounding box overlaps between frames
- Analyze trajectory direction and proximity changes
- Consider object size changes (approaching/receding)

CRITICAL RULES:
1. **DYNAMIC PRIORITY FOR MOBILE OBJECTS**: For soldiers and other mobile objects (moving entities), PRIORITIZE dynamic relations over static relations. If a soldier is moving, focus on extracting "enter", "exit", or "pass by" relationships.

2. Dynamic Relation Inference: Infer dynamic predicates by analyzing changes between the previous and current frames:
   - For 'enter'/'exit': Check if object bounding boxes cross boundaries or overlap changes
   - For 'pass by': Detect trajectory changes and proximity variations without boundary crossing

3. Static Relation Persistence: Assume previous static relations persist unless there is a clear change in the current frame.

4. Logical Subject-Object Order: For dynamic predicates, the 'subject' MUST be the moving object (usually soldier).

5. STRICT ID-Label Consistency: IDs and labels in relations MUST PERFECTLY MATCH the provided object lists.

6. Categorized Output: The final output MUST be a JSON object with two keys: `static_relations` and `dynamic_relations`.

7. **SOLDIER AND BUILDING MOVEMENT FOCUS**: Pay special attention to soldier and building movements. If any soldier shows position change, displacement, or trajectory variation, prioritize generating dynamic relations for that soldier.

Example of a valid JSON output:
{{
  "static_relations": [
    {{
      "subject_id": "5", "predicate": "next to", "object_id": "6", "subject_label": "car", "object_label": "building"
    }}
  ],
  "dynamic_relations": [
    {{
      "subject_id": "1", "predicate": "enter", "object_id": "2", "subject_label": "soldier", "object_label": "building"
    }},
    {{
      "subject_id": "3", "predicate": "pass by", "object_id": "4", "subject_label": "soldier", "object_label": "car"
    }}
  ]
}}

TASK
1. From the provided frame(s) and object list, output a JSON object with two keys only:
   • "static_relations"   – max 5 items  
   • "dynamic_relations" – max 5 items  
2. **PRIORITIZE dynamic relations for soldiers and mobile objects over static relations**
3. Rank candidate relations by visual certainty and overall relevance, then keep only the top five in each list.
4. Follow these formatting rules exactly:
   • Each relation must be an object with the keys:
     "subject_id", "predicate", "object_id", "subject_label", "object_label"
   • If fewer than five valid relations exist, return only those found.
   • Do not add any keys or metadata outside the required JSON structure.
5. Allowed predicates and objects remain the same as previously specified.

Return **only** the final valid JSON
"""


/home/etri/miniconda3/envs/test_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
import re

In [2]:
# --- 유틸리티 함수 ---
def load_json_data(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            return json.load(f)
    except Exception as e:
        print(f"JSON 파일 로드 오류: {e}")
        return None

def load_frame_image(folder_path, video_name, frame_number):
    image_filename = f"{video_name}_frame_{frame_number:04d}.png"
    image_path = os.path.join(folder_path, image_filename)
    try:
        image = Image.open(image_path).convert("RGB")
        return image
    except FileNotFoundError:
        return None

# --- 불완전 JSON 복구 함수 (새 버전: 마지막 '}, {' 기준으로 자르고 '}]}' 붙임) ---
def repair_incomplete_json(response_text):
    # 코드 블록 제거 (예: ``````)
    response_text = re.sub(r'^\s*``````\s*$', '', response_text.strip())
    
    # 가장 큰 JSON 객체 추출
    json_match = re.search(r'\{[\s\S]*\}', response_text)
    if not json_match:
        return None
    text = json_match.group(0)
    
    # 마지막 '}, {' 위치 찾기
    pattern = r"}\s*,\s*{"
    matches = list(re.finditer(pattern, text))
    if not matches:
        # 패턴 없으면 원본 파싱 시도
        try:
            return json.loads(text)
        except json.JSONDecodeError:
            return None
    
    last_match = matches[-1]
    last_pos = last_match.start() + 1  # '}' 위치 (마지막 완전한 객체의 끝)
    
    # 마지막 완전한 객체까지만 자르고 배열/객체 닫기
    repaired = text[:last_pos] + ']}'
    
    # 파싱 시도
    try:
        parsed = json.loads(repaired)
        return parsed
    except json.JSONDecodeError as e:
        print(f"JSON 복구 후 파싱 오류: {e}")
        return None

In [8]:
# --- 모델 및 프로세서 로드 ---
print("모델과 프로세서를 로드합니다...")
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2.5-VL-7B-Instruct",
    torch_dtype="auto",
    device_map="auto"
)
processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-7B-Instruct")
print("로드 완료.")

# --- 메인 로직 ---
video_data = load_json_data(json_file_path)
if not video_data:
    exit()

updated_frames = []

for i, frame_info in enumerate(video_data):
    frame_number = frame_info['frame']
    objects_list = frame_info['objects']
    objects_str = json.dumps(objects_list, indent=4)
    
    print(f"--- 프레임 {frame_number} 처리 시작 ---")

    messages = [{"role": "user", "content": []}]

    if i == 0:
        current_image = load_frame_image(frame_folder_path, video_name, frame_number)
        if current_image is None:
            print(f"프레임 {frame_number} 이미지를 찾을 수 없습니다.")
            continue
        
        prompt_text = base_prompt.format(
            frame_number=frame_number, total_frames=len(video_data),
            objects_str=objects_str, video_captioning_info=video_captioning_info
        )
        messages[0]["content"].append({"type": "image", "image": current_image})
        messages[0]["content"].append({"type": "text", "text": prompt_text})
    else:
        prev_frame_info = video_data[i-1]
        prev_frame_number = prev_frame_info['frame']
        
        prev_image = load_frame_image(frame_folder_path, video_name, prev_frame_number)
        current_image = load_frame_image(frame_folder_path, video_name, frame_number)

        if prev_image is None or current_image is None:
            print(f"프레임 {prev_frame_number} 또는 {frame_number} 이미지를 찾을 수 없습니다.")
            continue
            
        previous_objects_str = json.dumps(prev_frame_info.get('objects', []), indent=4)
        
        prompt_text = modified_prompt.format(
            frame_number=frame_number, previous_objects_str=previous_objects_str,
            objects_str=objects_str, video_captioning_info=video_captioning_info
        )
        
        messages[0]["content"].append({"type": "text", "text": "Analyze the following two images to extract relationships."})
        messages[0]["content"].append({"type": "image", "image": prev_image})
        messages[0]["content"].append({"type": "image", "image": current_image})
        messages[0]["content"].append({"type": "text", "text": prompt_text})

    extracted_relations = {"static_relations": [], "dynamic_relations": []}
    try:
        text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        
        image_inputs, video_inputs = process_vision_info(messages)
        
        # 디버깅: 입력 길이 출력
        print(f"프레임 {frame_number} - image_inputs 길이: {len(image_inputs) if image_inputs else 0}, video_inputs 길이: {len(video_inputs) if video_inputs else 0}")
        
        inputs = processor(
            text=[text],
            images=image_inputs,
            videos=video_inputs,
            padding=True,
            return_tensors="pt"
        ).to(model.device)
        
        with torch.no_grad():
            generated_ids = model.generate(
                **inputs,
                max_new_tokens=1024,
                do_sample=False
            )
        
        response_text_list = processor.batch_decode(generated_ids[:, inputs.input_ids.shape[-1]:], skip_special_tokens=True)
        
        if response_text_list:
            response_text = response_text_list[0]
            print(f"response_text : {response_text}")
            print(f"프레임 {frame_number} - 생성된 응답 길이: {len(response_text)} 자")
            
            # [수정됨] 새로운 복구 함수 호출
            repaired_json = repair_incomplete_json(response_text)
            if repaired_json:
                extracted_relations = repaired_json
            else:
                print(f"프레임 {frame_number}: JSON 복구 실패. 원본 응답: \n{response_text}")
        else:
            print(f"프레임 {frame_number}: 모델에서 빈 응답을 받았습니다.")

    except Exception as e:
        print(f"프레임 {frame_number} 처리 중 오류 발생: {e}")

    frame_info['static_relations'] = extracted_relations.get('static_relations', [])
    frame_info['dynamic_relations'] = extracted_relations.get('dynamic_relations', [])
    updated_frames.append(frame_info)
    
    print(f"프레임 {frame_number} 관계 추출 완료. Static: {len(frame_info['static_relations'])}, Dynamic: {len(frame_info['dynamic_relations'])}")

with open(output_json_path, 'w', encoding='utf-8') as f:
    json.dump(updated_frames, f, indent=4, ensure_ascii=False)

print(f"\n모든 작업 완료. 결과가 '{output_json_path}'에 저장되었습니다.")

모델과 프로세서를 로드합니다...


Loading checkpoint shards: 100%|██████████| 5/5 [00:05<00:00,  1.07s/it]
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


로드 완료.
--- 프레임 0 처리 시작 ---
프레임 0 - image_inputs 길이: 1, video_inputs 길이: 0
response_text : ```json
{
  "static_relations": [
    {
      "subject_id": "1",
      "predicate": "next to",
      "object_id": "2",
      "subject_label": "palm tree",
      "object_label": "building"
    },
    {
      "subject_id": "2",
      "predicate": "next to",
      "object_id": "3",
      "subject_label": "palm tree",
      "object_label": "building"
    },
    {
      "subject_id": "3",
      "predicate": "next to",
      "object_id": "4",
      "subject_label": "palm tree",
      "object_label": "building"
    },
    {
      "subject_id": "4",
      "predicate": "next to",
      "object_id": "5",
      "subject_label": "palm tree",
      "object_label": "building"
    },
    {
      "subject_id": "6",
      "predicate": "next to",
      "object_id": "7",
      "subject_label": "palm tree",
      "object_label": "building"
    }
  ],
  "dynamic_relations": []
}
```
프레임 0 - 생성된 응답 길이: 874 자
프레임 0 관계 추

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


프레임 1 - image_inputs 길이: 2, video_inputs 길이: 0
response_text : ```json
{
  "static_relations": [],
  "dynamic_relations": [
    {
      "subject_id": "1",
      "predicate": "pass by",
      "object_id": "2",
      "subject_label": "soldier",
      "object_label": "building"
    },
    {
      "subject_id": "1",
      "predicate": "pass by",
      "object_id": "3",
      "subject_label": "soldier",
      "object_label": "car"
    }
  ]
}
```
프레임 1 - 생성된 응답 길이: 382 자
프레임 1 관계 추출 완료. Static: 0, Dynamic: 1
--- 프레임 2 처리 시작 ---


...

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


프레임 21 - image_inputs 길이: 2, video_inputs 길이: 0
response_text : ```json
{
  "static_relations": [
    {
      "subject_id": "7",
      "predicate": "next to",
      "object_id": "3",
      "subject_label": "soldier",
      "object_label": "car"
    },
    {
      "subject_id": "8",
      "predicate": "next to",
      "object_id": "3",
      "subject_label": "soldier",
      "object_label": "car"
    },
    {
      "subject_id": "9",
      "predicate": "next to",
      "object_id": "3",
      "subject_label": "soldier",
      "object_label": "car"
    },
    {
      "subject_id": "10",
      "predicate": "next to",
      "object_id": "3",
      "subject_label": "soldier",
      "object_label": "car"
    },
    {
      "subject_id": "11",
      "predicate": "next to",
      "object_id": "3",
      "subject_label": "soldier",
      "object_label": "car"
    }
  ],
  "dynamic_relations": [
    {
      "subject_id": "7",
      "predicate": "pass by",
      "object_id": "3",
      "subject_l

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


프레임 22 - image_inputs 길이: 2, video_inputs 길이: 0


KeyboardInterrupt: 

In [9]:
import torch
from transformers import Qwen2_5_VLForConditionalGeneration, AutoProcessor
from qwen_vl_utils import process_vision_info
import json
import os
from PIL import Image

# --- 모델 및 프로세서 로드 ---
print("모델과 프로세서를 로드합니다...")
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2.5-VL-7B-Instruct",
    torch_dtype="auto",
    device_map="auto"
)
processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-7B-Instruct")
print("로드 완료.")

# --- 설정 ---
json_file_path = '/home/etri/DHL/Data/test_data/건물진입_tracking.json'
frame_folder_path = '/home/etri/DHL/Data/test_data/output_frames_1'
video_name = '건물진입'
output_json_path = '/home/etri/DHL/SGG+LLM/output_relation/qwen_output_relations_final_final_final.json'
video_captioning_info = "A group of soldiers is seen running down a deserted street. After running down a deserted street, they are entering the building in a tropical setting.The environment features palm trees, small buildings with flat roofs, and a dirt hillside in the background. A blue car is parked on the side of the road, and a red barrier is placed near."

# --- 수정된 프롬프트 템플릿 ---
base_prompt = """
You are an expert in video analysis and extracting scene graph triplets for military and tactical scenarios.
Your primary goal is to extract the maximum number of static relationships possible between all detected objects. You must consider every plausible static relationship for all object pairs.
Your task is to identify ALL relevant static relationships between these objects and output them as a single JSON object containing two separate lists: `static_relations` and `dynamic_relations`.
The 'predicate' should describe the relationship accurately based on the visual context, the detected objects, and the provided captioning information.

This is the very first frame of the video (Frame {frame_number} out of {total_frames}).
Here are the detected objects in this frame:
{objects_str}

Overall Context: "{video_captioning_info}"

ALLOWED OBJECTS:
soldier, building

ALLOWED PREDICATES:
1. Static Relations: next to, in front of, on top of, under, covered by, inside, outside, between
2. Dynamic Relations: enter, exit, pass by

CRITICAL RULES:
1. STRICT ID-Label Consistency: The `subject_id` and `object_id` MUST PERFECTLY MATCH the `track_id`. `subject_label` and `object_label` MUST MATCH the `label`.
2. Categorized Output: The final output MUST be a JSON object with two keys: `static_relations` and `dynamic_relations`. If none, output empty lists.
3. For this first frame, only `static_relations` should be populated.
4. You must use ALLOWED OBJECTS

TASK:
1. Analyze the video frames and bounding box data
2. Output a JSON object with two keys: "static_relations" and "dynamic_relations" 
3. Maximum 5 items per category


Example of a valid JSON output:
{{
  "static_relations": [
    {{
      "subject_id": "5", "predicate": "next to", "object_id": "6", "subject_label": "car", "object_label": "building"
    }}
  ],
  "dynamic_relations": [
    {{
      "subject_id": "1", "predicate": "enter", "object_id": "2", "subject_label": "soldier", "object_label": "building"
    }},
    {{
      "subject_id": "3", "predicate": "pass by", "object_id": "4", "subject_label": "soldier", "object_label": "building"
    }}
  ]
}}

Return **only** the final valid JSON
"""

# 비디오 입력을 위한 새로운 프롬프트 (2프레임 비디오 + 이전 프레임 바운딩박스 정보)
video_prompt = """
You are an expert in video analysis and extracting scene graph triplets for military and tactical scenarios.
Your task is to analyze consecutive frames from a video to identify both static and dynamic relationships between objects.
This is frame {frame_number} out of {total_frames} in the video.
You are provided with:
1. A video containing 2 frames (previous frame and current frame)
2. Bounding box information from an earlier frame for context

Context Information:
- Earlier Frame ({earlier_frame_number}) Objects: {earlier_objects_str}
- Current Frame ({frame_number}) Objects: {objects_str}
- Overall context: "{video_captioning_info}"

**ALLOWED OBJECTS:**
soldier, building

**ALLOWED PREDICATES:**
1. Static Relations: next to, in front of, on top of, under, covered by, inside, outside, between
2. Dynamic Relations: enter, exit, pass by

ANALYSIS GUIDELINES:
1. **Motion Analysis**: Compare the two video frames to detect object movements and trajectory changes
2. **Contextual Understanding**: Use the earlier frame bounding box data to understand the broader movement patterns
3. **Dynamic Priority**: Prioritize dynamic relations (enter, exit, pass by) for moving objects like soldiers
4. **Static Relations**: Extract static relations based on the current frame's spatial arrangements

DYNAMIC RELATIONS DEFINITIONS:
• "enter": Object moves INTO another object's boundary or space

{{"subject_id": "1", "predicate": "enter", "object_id": "2", "subject_label": "soldier", "object_label": "building"}}

• "exit": Object moves OUT OF another object's boundary or space

{{"subject_id": "1", "predicate": "exit", "object_id": "2", "subject_label": "soldier", "object_label": "building"}} 

• "pass by": Object moves NEAR another object without entering
{{"subject_id": "1", "predicate": "pass by", "object_id": "3", "subject_label": "soldier", "object_label": "building"}}
{{"subject_id": "1", "predicate": "pass by", "object_id": "3", "subject_label": "soldier", "object_label": "car"}}

**CRITICAL RULES:**
1. STRICT ID-Label Consistency: IDs and labels MUST match the provided object lists
2. Dynamic Priority: For soldiers, prioritize dynamic relations over static relations
3. Spatial Proximity: Only extract static relations and dynamic relations for the nearest object. (adjacent/overlapping bounding boxes)
4. Movement Focus: If an object shows position changes between frames, generate dynamic relations
5. You must use ALLOWED OBJECTS
6. **PRIORITY DYNAMIC RELATIONS**: You must focus specifically on detecting dynamic relations between these critical object pairs:
   
   **HIGH PRIORITY PAIRS:**
   - (soldier, building): Detect "enter" (soldier moving into building), "exit" (soldier leaving building), "pass by" (soldier moving near building)
   - (soldier, car): Detect "enter" (soldier getting into car), "exit" (soldier getting out of car), "pass by" (soldier moving past car)

TASK:
1. Analyze the video frames and bounding box data
2. Output a JSON object with two keys: "static_relations" and "dynamic_relations" 

Example of a valid JSON output:
{{
  "static_relations": [
    {{
      "subject_id": "5", "predicate": "next to", "object_id": "6", "subject_label": "car", "object_label": "building"
    }}
  ],
  "dynamic_relations": [
    {{
      "subject_id": "1", "predicate": "enter", "object_id": "2", "subject_label": "soldier", "object_label": "building"
    }},
    {{
      "subject_id": "3", "predicate": "pass by", "object_id": "4", "subject_label": "soldier", "object_label": "building"
    }}
  ]
}}

Return **only** the final valid JSON
"""

# --- 메인 로직 ---
video_data = load_json_data(json_file_path)
if not video_data:
    exit()

updated_frames = []

for i, frame_info in enumerate(video_data):
    frame_number = frame_info['frame']
    objects_list = frame_info['objects']
    objects_str = json.dumps(objects_list, indent=4)
    
    print(f"--- 프레임 {frame_number} 처리 시작 ---")

    if i == 0:
        # 첫 번째 프레임: 단일 이미지 처리
        current_image = load_frame_image(frame_folder_path, video_name, frame_number)
        if current_image is None:
            print(f"프레임 {frame_number} 이미지를 찾을 수 없습니다.")
            continue
        
        prompt_text = base_prompt.format(
            frame_number=frame_number, total_frames=len(video_data),
            objects_str=objects_str, video_captioning_info=video_captioning_info
        )
        
        messages = [
            {
                "role": "user",
                "content": [
                    {"type": "image", "image": current_image},
                    {"type": "text", "text": prompt_text}
                ]
            }
        ]
    else:
        # 두 번째 프레임부터: 비디오 입력 (이전 프레임 + 현재 프레임) + 더 이전 프레임의 바운딩박스 정보
        prev_frame_info = video_data[i-1]
        prev_frame_number = prev_frame_info['frame']
        
        # 비디오 파일 경로 생성 (이전 프레임 + 현재 프레임)
        prev_frame_path = f"file://{frame_folder_path}/{video_name}_frame_{prev_frame_number:04d}.png"
        current_frame_path = f"file://{frame_folder_path}/{video_name}_frame_{frame_number:04d}.png"
        
        # 파일 존재 여부 확인
        if not os.path.exists(prev_frame_path.replace("file://", "")) or not os.path.exists(current_frame_path.replace("file://", "")):
            print(f"프레임 {prev_frame_number} 또는 {frame_number} 이미지를 찾을 수 없습니다.")
            continue
        
        # 더 이전 프레임의 바운딩박스 정보 (있는 경우)
        if i >= 2:
            earlier_frame_info = video_data[i-2]
            earlier_frame_number = earlier_frame_info['frame']
            earlier_objects_str = json.dumps(earlier_frame_info.get('objects', []), indent=4)
        else:
            # 이전 프레임이 없는 경우 첫 번째 프레임 정보 사용
            earlier_frame_info = video_data[0]
            earlier_frame_number = earlier_frame_info['frame']
            earlier_objects_str = json.dumps(earlier_frame_info.get('objects', []), indent=4)
        
        prompt_text = video_prompt.format(
            frame_number=frame_number,
            earlier_frame_number=earlier_frame_number,
            earlier_objects_str=earlier_objects_str,
            objects_str=objects_str,
            video_captioning_info=video_captioning_info,
            total_frames=len(video_data)
        )
        
        # 비디오 입력을 위한 메시지 구조
        messages = [
            {
                "role": "user",
                "content": [
                    {
                        "type": "video",
                        "video": [
                            prev_frame_path,
                            current_frame_path
                        ],
                    },
                    {"type": "text", "text": prompt_text}
                ],
            }
        ]

    extracted_relations = {"static_relations": [], "dynamic_relations": []}
    try:
        text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        
        image_inputs, video_inputs = process_vision_info(messages)
        
        # 디버깅: 입력 길이 출력
        print(f"프레임 {frame_number} - image_inputs 길이: {len(image_inputs) if image_inputs else 0}, video_inputs 길이: {len(video_inputs) if video_inputs else 0}")
        
        inputs = processor(
            text=[text],
            images=image_inputs,
            videos=video_inputs,
            padding=True,
            return_tensors="pt"
        ).to(model.device)
        
        with torch.no_grad():
            generated_ids = model.generate(
                **inputs,
                max_new_tokens=1024,
                do_sample=False
            )
        
        response_text_list = processor.batch_decode(generated_ids[:, inputs.input_ids.shape[-1]:], skip_special_tokens=True)
        
        if response_text_list:
            response_text = response_text_list[0]
            print(f"response_text : {response_text}")
            print(f"프레임 {frame_number} - 생성된 응답 길이: {len(response_text)} 자")
            
            # JSON 복구 함수 호출
            repaired_json = repair_incomplete_json(response_text)
            if repaired_json:
                extracted_relations = repaired_json
            else:
                print(f"프레임 {frame_number}: JSON 복구 실패. 원본 응답: \n{response_text}")
        else:
            print(f"프레임 {frame_number}: 모델에서 빈 응답을 받았습니다.")

    except Exception as e:
        print(f"프레임 {frame_number} 처리 중 오류 발생: {e}")

    frame_info['static_relations'] = extracted_relations.get('static_relations', [])
    frame_info['dynamic_relations'] = extracted_relations.get('dynamic_relations', [])
    updated_frames.append(frame_info)
    
    print(f"프레임 {frame_number} 관계 추출 완료. Static: {len(frame_info['static_relations'])}, Dynamic: {len(frame_info['dynamic_relations'])}")

with open(output_json_path, 'w', encoding='utf-8') as f:
    json.dump(updated_frames, f, indent=4, ensure_ascii=False)

print(f"\n모든 작업 완료. 결과가 '{output_json_path}'에 저장되었습니다.")


모델과 프로세서를 로드합니다...


Loading checkpoint shards: 100%|██████████| 5/5 [00:05<00:00,  1.15s/it]


로드 완료.
--- 프레임 0 처리 시작 ---
프레임 0 - image_inputs 길이: 1, video_inputs 길이: 0


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


response_text : ```json
{
  "static_relations": [
    {
      "subject_id": "1",
      "predicate": "next to",
      "object_id": "2",
      "subject_label": "soldier",
      "object_label": "building"
    },
    {
      "subject_id": "3",
      "predicate": "next to",
      "object_id": "4",
      "subject_label": "soldier",
      "object_label": "building"
    },
    {
      "subject_id": "5",
      "predicate": "next to",
      "object_id": "6",
      "subject_label": "car",
      "object_label": "building"
    },
    {
      "subject_id": "7",
      "predicate": "next to",
      "object_id": "8",
      "subject_label": "barrier",
      "object_label": "building"
    },
    {
      "subject_id": "9",
      "predicate": "next to",
      "object_id": "10",
      "subject_label": "palm tree",
      "object_label": "building"
    }
  ],
  "dynamic_relations": []
}
```
프레임 0 - 생성된 응답 길이: 863 자
프레임 0 관계 추출 완료. Static: 4, Dynamic: 0
--- 프레임 1 처리 시작 ---


...

프레임 115 - image_inputs 길이: 0, video_inputs 길이: 1
response_text : ```json
{
  "static_relations": [
    {
      "subject_id": "5", "predicate": "next to", "object_id": "6", "subject_label": "building", "object_label": "building"
    },
    {
      "subject_id": "3", "predicate": "next to", "object_id": "4", "subject_label": "car", "object_label": "building"
    }
  ],
  "dynamic_relations": [
    {
      "subject_id": "19", "predicate": "pass by", "object_id": "12", "subject_label": "soldier", "object_label": "building"
    },
    {
      "subject_id": "26", "predicate": "pass by", "object_id": "12", "subject_label": "soldier", "object_label": "building"
    },
    {
      "subject_id": "24", "predicate": "pass by", "object_id": "12", "subject_label": "soldier", "object_label": "building"
    }
  ]
}
```
프레임 115 - 생성된 응답 길이: 749 자
프레임 115 관계 추출 완료. Static: 2, Dynamic: 2
--- 프레임 116 처리 시작 ---


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


프레임 116 - image_inputs 길이: 0, video_inputs 길이: 1
response_text : ```json
{
  "static_relations": [
    {
      "subject_id": "5", "predicate": "next to", "object_id": "6", "subject_label": "building", "object_label": "building"
    },
    {
      "subject_id": "3", "predicate": "next to", "object_id": "4", "subject_label": "car", "object_label": "building"
    }
  ],
  "dynamic_relations": [
    {
      "subject_id": "9", "predicate": "pass by", "object_id": "12", "subject_label": "soldier", "object_label": "building"
    },
    {
      "subject_id": "10", "predicate": "pass by", "object_id": "12", "subject_label": "soldier", "object_label": "building"
    },
    {
      "subject_id": "19", "predicate": "pass by", "object_id": "12", "subject_label": "soldier", "object_label": "building"
    },
    {
      "subject_id": "24", "predicate": "pass by", "object_id": "12", "subject_label": "soldier", "object_label": "building"
    },
    {
      "subject_id": "26", "predicate": "pass by", "o

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


프레임 117 - image_inputs 길이: 0, video_inputs 길이: 1
